In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm


train_path = "/content/drive/My Drive/SKIN CANCER DATASET/Train"
test_path = "/content/drive/My Drive/SKIN CANCER DATASET/Test"


output_path = "/content/drive/My Drive/SKIN CANCER DATASET/preprocessed_images"
os.makedirs(output_path, exist_ok=True)


def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(1, (17, 17))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    result = cv2.inpaint(img, mask, 1, cv2.INPAINT_TELEA)


    lab = cv2.cvtColor(result, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    enhanced = cv2.merge((cl, a, b))
    final = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

    return final


folders = [train_path, test_path]
count = 0

for folder in folders:
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        if os.path.isdir(subfolder_path):
            for image_name in tqdm(os.listdir(subfolder_path), desc=f"Processing {subfolder}"):
                image_path = os.path.join(subfolder_path, image_name)
                try:
                    preprocessed_img = preprocess_image(image_path)
                    save_path = os.path.join(output_path, f"{subfolder}_{count}.jpg")
                    cv2.imwrite(save_path, preprocessed_img)
                    count += 1
                except Exception as e:
                    print(f" Error processing {image_path}: {e}")

print(f"\n Total {count} preprocessed images saved in {output_path}")


Processing basal cell carcinoma: 100%|██████████| 16/16 [00:05<00:00,  2.68it/s]


✅ Total 2357 preprocessed images saved in /content/drive/My Drive/SKIN CANCER DATASET/preprocessed_images
